# Dataset Information & Automatic Download

This notebook uses sample histopathology images from open datasets. The images will be automatically downloaded to the `data/` folder if they don't exist.

**Datasets Used:**
- **CAMELYON16**: Lymph node sections for metastasis detection
- **Sample WSI files**: Representative whole slide images for demonstration
- **Test images**: Smaller patches for quick processing

**Data Sources:**
- CAMELYON16 Challenge: https://camelyon16.grand-challenge.org/
- OpenSlide Test Data: https://openslide.cs.cmu.edu/demo/
- Kaggle Histopathologic Cancer Detection: Sample patches

In [ ]:
import os
import urllib.request
from pathlib import Path
import zipfile
import requests
from tqdm import tqdm

def create_data_directory():
    """Create data directory if it doesn't exist"""
    data_dir = Path("../data")
    data_dir.mkdir(parents=True, exist_ok=True)
    return data_dir

def download_file_with_progress(url, filename, description="Downloading"):
    """Download file with progress bar"""
    try:
        response = requests.get(url, stream=True)
        response.raise_for_status()
        
        total_size = int(response.headers.get('content-length', 0))
        
        with open(filename, 'wb') as file, tqdm(
            desc=description,
            total=total_size,
            unit='B',
            unit_scale=True,
            unit_divisor=1024,
        ) as pbar:
            for chunk in response.iter_content(chunk_size=8192):
                if chunk:
                    file.write(chunk)
                    pbar.update(len(chunk))
        
        return True
    except Exception as e:
        print(f"Error downloading {filename}: {e}")
        return False

def download_sample_wsi():
    """Download sample WSI files for the tutorial"""
    data_dir = create_data_directory()
    
    # Sample files from OpenSlide demo data
    sample_files = {
        'CMU-1-Small-Region.svs': 'https://openslide.cs.cmu.edu/download/openslide-testdata/CMU-1-Small-Region.svs',
        'sample_patch_1.jpg': 'https://github.com/jamesdolezal/slideflow/raw/master/docs/source/_static/img/tile_1.jpg',
        'sample_patch_2.jpg': 'https://github.com/jamesdolezal/slideflow/raw/master/docs/source/_static/img/tile_2.jpg'
    }
    
    downloaded_files = []
    
    for filename, url in sample_files.items():
        file_path = data_dir / filename
        
        if file_path.exists():
            print(f"✅ {filename} already exists")
        else:
            print(f"📥 Downloading {filename}...")
            if download_file_with_progress(url, file_path, f"Downloading {filename}"):
                downloaded_files.append(filename)
                print(f"✅ Successfully downloaded {filename}")
            else:
                print(f"❌ Failed to download {filename}")
    
    return data_dir, downloaded_files

# Download the datasets
print("🚀 Initializing dataset download...")
data_directory, new_downloads = download_sample_wsi()
print(f"📁 Data directory: {data_directory.absolute()}")
print(f"🎯 Downloaded {len(new_downloads)} new files")

## Getting Started: Loading and Viewing a Digital Slide

This notebook covers the essential first step in any computational pathology project: accessing and displaying a whole-slide image (WSI).

**Our Goals:**
1.  Set up the environment by installing and importing key libraries.
2.  Use the `openslide-python` library to open a WSI file.
3.  Examine the slide's metadata, such as its different magnification levels.
4.  Extract a small tile (a patch) from the slide.
5.  Display the extracted tile using `matplotlib`.

### 1. Environment Setup

First, we need to install the necessary Python packages. We'll use `openslide-python` for handling WSI files, `Pillow` for image objects, and `matplotlib` for plotting.

Execute this command in your terminal to install them:
```bash
pip install openslide-python Pillow matplotlib
```
After installation, we can import them into our notebook.

In [ ]:
import openslide
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np
import os

# --- Configuration ---
# Use the downloaded sample WSI file
wsi_filepath = '../data/CMU-1-Small-Region.svs'

print("Libraries imported and file path configured.")
print(f"Target WSI file: {wsi_filepath}")

# Check if file exists, if not use sample patch images
if not os.path.exists(wsi_filepath):
    print(f"⚠️  WSI file not found, will use sample patches instead")
    sample_patches = ['../data/sample_patch_1.jpg', '../data/sample_patch_2.jpg']
    print(f"Sample patches: {sample_patches}")

### 2. Opening the WSI File

Let's use OpenSlide to create a slide object. This is memory-efficient as it doesn't load the entire gigapixel image at once. We'll include a check to ensure the file exists before trying to open it.

In [ ]:
slide_handle = None
if not os.path.exists(wsi_filepath):
    print(f"ERROR: File not found at {wsi_filepath}")
    print("Please ensure you have downloaded the data and the path is correct.")
else:
    try:
        slide_handle = openslide.OpenSlide(wsi_filepath)
        print(f"Successfully opened: {os.path.basename(wsi_filepath)}")
    except openslide.OpenSlideError as e:
        print(f"Failed to open slide. Details: {e}")

### 3. Exploring Slide Properties

WSIs are stored as image pyramids with multiple layers, each at a different resolution. Level 0 is the highest-resolution layer. Let's see what levels are available in our file.

In [ ]:
if slide_handle:
    print(f"Total levels: {slide_handle.level_count}")
    for level_index, dims in enumerate(slide_handle.level_dimensions):
        downsample_factor = slide_handle.level_downsamples[level_index]
        print(f"- Level {level_index}: Dimensions={dims}, Downsample={downsample_factor:.2f}x")

### 4. Extracting a Region of Interest

We can't work with the whole slide at once. Instead, we extract smaller tiles. The `read_region` method allows us to specify a location `(x, y)` in the level 0 coordinate system, the `level` to read from, and the desired `size` of the tile.

In [ ]:
# The top-left coordinate for our tile (from Level 0)
read_location = (50000, 35000)

# The pyramid level to read from
level_to_read = 0

# The desired tile size (width, height)
tile_dimensions = (256, 256)

tile_image = None
if slide_handle:
    tile_image = slide_handle.read_region(read_location, level_to_read, tile_dimensions)
    print(f"Extracted a {tile_image.size} tile from level {level_to_read}.")

### 5. Visualizing the Extracted Tile

`read_region` gives us a standard `Pillow` Image object, which we can easily plot.

In [ ]:
if tile_image:
    # The output from read_region is RGBA, let's convert to RGB for cleaner display
    rgb_tile = tile_image.convert('RGB')
    
    plt.figure(figsize=(6, 6))
    plt.imshow(rgb_tile)
    plt.title(f"Tile from Level {level_to_read} at {read_location}")
    plt.xlabel('X-axis (pixels)')
    plt.ylabel('Y-axis (pixels)')
    plt.grid(False)
    plt.show()

## ✅ Final Check

To complete this task, we need to verify that our extracted tile has the correct dimensions. The cell below converts the `Pillow` image to a `NumPy` array and asserts its shape.

In [ ]:
if 'rgb_tile' in locals() and rgb_tile is not None:
    image_as_array = np.array(rgb_tile)
    
    # The shape should be (height, width, channels)
    expected_shape = (256, 256, 3)
    assert image_as_array.shape == expected_shape, f"Shape mismatch! Expected {expected_shape}, but got {image_as_array.shape}"
    
    print("SUCCESS: The extracted image tile has the correct shape.")
    print(f"Shape: {image_as_array.shape}")
else:
    print("Skipping test: The image tile was not loaded correctly in previous steps.")